In [1]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import torch
import torch_geometric
from torch_geometric.data import Data


import torch
import torch_geometric
from torch_geometric.data import Data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [3]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)


cuda:0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
classes_filename = '/content/drive/MyDrive/Colab Notebooks/ssn/projekt/datasets/elliptic_bitcoin_dataset/elliptic_txs_classes.csv'
classes_dataframe = pd.read_csv(classes_filename, sep=',')
classes_dataframe

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown
...,...,...
203764,173077460,unknown
203765,158577750,unknown
203766,158375402,1
203767,158654197,unknown


In [6]:
edges_filename = '/content/drive/MyDrive/Colab Notebooks/ssn/projekt/datasets/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv'
edges_dataframe = pd.read_csv(edges_filename, sep=',')
edges_dataframe

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206
...,...,...
234350,158365409,157930723
234351,188708874,188708879
234352,157659064,157659046
234353,87414554,106877725


In [7]:
node_features_filename = '/content/drive/MyDrive/Colab Notebooks/ssn/projekt/datasets/elliptic_bitcoin_dataset/elliptic_txs_features.csv'
node_features_dataframe = pd.read_csv(node_features_filename, sep=',', header=None)
node_features_dataframe

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,173077460,49,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
203765,158577750,49,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,...,0.162722,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984
203766,158375402,49,-0.172014,-0.078182,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163626,...,1.261246,1.985050,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
203767,158654197,49,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,...,-0.397749,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399


In [8]:
classes_dataframe

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown
...,...,...
203764,173077460,unknown
203765,158577750,unknown
203766,158375402,1
203767,158654197,unknown


In [9]:
# rows with class 0 and 1 will be used for training
# rows with class 2 will be used for prediction


def remap_label(label):
  if label == '1': return 0
  if label == '2': return 1
  return 2

In [10]:

classes_dataframe_after_remap = classes_dataframe.copy()
classes_dataframe_after_remap['class'] = classes_dataframe['class'].map(remap_label)
classes_dataframe_after_remap['class']



0         2
1         2
2         2
3         1
4         2
         ..
203764    2
203765    2
203766    0
203767    2
203768    2
Name: class, Length: 203769, dtype: int64

In [27]:

node_features_dataframe_with_class = node_features_dataframe.copy()
node_features_dataframe_with_class['class'] = classes_dataframe_after_remap['class']
node_features_dataframe_with_class

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,class
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,1
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,173077460,49,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,...,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
203765,158577750,49,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,...,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984,2
203766,158375402,49,-0.172014,-0.078182,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163626,...,1.985050,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0
203767,158654197,49,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,...,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399,2


In [28]:
node_features_dataframe_with_class[0]

0         230425980
1           5530458
2         232022460
3         232438397
4         230460314
            ...    
203764    173077460
203765    158577750
203766    158375402
203767    158654197
203768    157597225
Name: 0, Length: 203769, dtype: int64

In [29]:
# lets reindex the node ids so that node with index 0 has id 0 instead of 230425980, this way we will be able to drop column 0 of node_features_dataframe

reindex_dictionary = dict()
for i, node_id in enumerate(node_features_dataframe_with_class[0]):
  reindex_dictionary[node_id] = i
  
node_features_dataframe_with_class_and_remapped_node_id = node_features_dataframe_with_class.copy()
node_features_dataframe_with_class_and_remapped_node_id[0] = node_features_dataframe_with_class[0].map(lambda node_id: reindex_dictionary[node_id])
node_features_dataframe_with_class_and_remapped_node_id

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,class
0,0,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
1,1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
2,2,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2
3,3,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,1
4,4,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,203764,49,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,...,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
203765,203765,49,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,...,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984,2
203766,203766,49,-0.172014,-0.078182,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163626,...,1.985050,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0
203767,203767,49,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,...,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399,2


In [14]:
# now we need to map the node_ids in edges_dataframe:
edges_dataframe


,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206
...,...,...
234350,158365409,157930723
234351,188708874,188708879
234352,157659064,157659046
234353,87414554,106877725


In [16]:
edges_dataframe_reindexed = edges_dataframe.copy()
edges_dataframe_reindexed['txId1'] = edges_dataframe['txId1'].map(lambda node_id: reindex_dictionary[node_id])
edges_dataframe_reindexed['txId2'] = edges_dataframe['txId2'].map(lambda node_id: reindex_dictionary[node_id])
edges_dataframe_reindexed



,txId1,txId2
0,0,1
1,2,3
2,4,5
3,6,7
4,8,9
...,...,...
234350,203602,201430
234351,203603,203099
234352,201921,202042
234353,201480,201368


In [86]:
 start_node_id = 2
#  np.array(node_features_dataframe_with_class_and_remapped_node_id[ node_features_dataframe_with_class_and_remapped_node_id[0] == start_node_id]['class'] == 2)[0]
# (node_features_dataframe_with_class_and_remapped_node_id.iloc[[start_node_id]]['class'] == 2)[0]

(node_features_dataframe_with_class_and_remapped_node_id.iloc[[start_node_id]]['class'] == 2).values[0]

True

In [142]:
# split the edge_dataframe into train and test sets:
# pd.DataFrame(np.array(edges_dataframe_reindexed))
train_edges = []
test_edges = []


# this may take some time...
for i, edge in enumerate(np.array(edges_dataframe_reindexed)):
  is_start_node_unknown = (node_features_dataframe_with_class_and_remapped_node_id.iloc[[edge[0]]]['class'] == 2).values[0]
  # is_start_node_unknown or is_end_node_unknown, i did this trick to allow for short circuit
  if is_start_node_unknown or (node_features_dataframe_with_class_and_remapped_node_id.iloc[[edge[1]]]['class'] == 2).values[0]: 
    test_edges.append(edge)
  else:
    train_edges.append(edge)

  # if i % 100000 == 0:
  #   print(i)
  # if i > 10: break

# print(train_edges)
# print(test_edges)

In [143]:
print(len(train_edges))
print(len(test_edges))
train_edges = pd.DataFrame(train_edges)
test_edges = pd.DataFrame(test_edges)
train_edges

36624
197731


,0,1
0,10,11
1,16,17
2,30,31
3,52,53
4,66,67
...,...,...
36619,202182,201458
36620,202183,201342
36621,201882,203604
36622,203604,201331


In [144]:
node_features_dataframe_with_class_and_remapped_node_id

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,class
0,0,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
1,1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
2,2,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2
3,3,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,1
4,4,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,203764,49,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,...,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
203765,203765,49,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,...,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984,2
203766,203766,49,-0.172014,-0.078182,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163626,...,1.985050,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0
203767,203767,49,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,...,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399,2


In [145]:
node_features_dataframe_with_class_without_timestep = node_features_dataframe_with_class_and_remapped_node_id.copy()

# removing column 1 which is timestep (nodes are only connected to nodes with the same timestep) since it is redundant. The network will know the which nodes are connected based on the edges_dataframe.
node_features_dataframe_with_class_without_timestep = node_features_dataframe_with_class_without_timestep.drop(columns=[0, 1])
# node_features_dataframe_with_class_without_timestep = node_features_dataframe_with_class_without_timestep.drop(columns=[1])
node_features_dataframe_with_class_without_timestep

,2,3,4,5,6,7,8,9,10,11,...,158,159,160,161,162,163,164,165,166,class
0,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,-0.049707,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,-0.049707,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
2,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,-0.049707,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2
3,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,0.043598,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,1
4,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,0.935886,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,-0.142008,-0.049707,...,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
203765,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,-0.162334,-0.049707,...,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984,2
203766,-0.172014,-0.078182,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163626,-0.168778,-0.045389,...,1.985050,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0
203767,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,-0.169317,-0.049707,...,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399,2


In [146]:
# split the node_features into train and test sets:
train_node_features = node_features_dataframe_with_class_without_timestep[node_features_dataframe_with_class_without_timestep['class'] != 2]
test_node_features = node_features_dataframe_with_class_without_timestep[node_features_dataframe_with_class_without_timestep['class'] == 2]

In [147]:
train_node_features

,2,3,4,5,6,7,8,9,10,11,...,158,159,160,161,162,163,164,165,166,class
3,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,0.043598,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,1
9,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,-0.163645,-0.144554,0.020069,...,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792,1
10,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.137933,-0.144108,-0.049707,...,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,1
11,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.141519,-0.147643,-0.049707,...,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,1
16,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,-0.163640,-0.169115,-0.047227,...,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203752,-0.159293,-0.037276,1.018602,-0.121970,0.035526,-0.113002,-0.061584,-0.149635,-0.155646,-0.049707,...,1.408971,0.231244,-0.388216,-0.098889,1.931078,3.168259,3.707301,-1.390548,-1.214035,1
203754,-0.172962,-0.126566,1.018602,-0.121970,-0.063725,-0.113002,-0.061584,-0.163622,-0.169437,-0.049707,...,0.647874,0.241128,0.241406,10.914916,1.700384,-0.131155,7.914145,-0.120613,-0.119792,1
203759,-0.170412,-0.078164,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163631,-0.167106,-0.032993,...,1.606604,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0
203763,-0.093732,-0.116160,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.082559,-0.089510,-0.049707,...,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,1


In [148]:
train_classes = train_node_features['class']
train_classes

3         1
9         1
10        1
11        1
16        1
         ..
203752    1
203754    1
203759    0
203763    1
203766    0
Name: class, Length: 46564, dtype: int64

In [149]:
train_node_features_without_class = train_node_features.drop(columns=['class'])
train_node_features_without_class

,2,3,4,5,6,7,8,9,10,11,...,157,158,159,160,161,162,163,164,165,166
3,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,0.043598,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
9,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,-0.163645,-0.144554,0.020069,...,-0.577099,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792
10,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.137933,-0.144108,-0.049707,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
11,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.141519,-0.147643,-0.049707,...,-0.539735,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
16,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,-0.163640,-0.169115,-0.047227,...,-0.577099,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203752,-0.159293,-0.037276,1.018602,-0.121970,0.035526,-0.113002,-0.061584,-0.149635,-0.155646,-0.049707,...,1.793987,1.408971,0.231244,-0.388216,-0.098889,1.931078,3.168259,3.707301,-1.390548,-1.214035
203754,-0.172962,-0.126566,1.018602,-0.121970,-0.063725,-0.113002,-0.061584,-0.163622,-0.169437,-0.049707,...,-0.577099,0.647874,0.241128,0.241406,10.914916,1.700384,-0.131155,7.914145,-0.120613,-0.119792
203759,-0.170412,-0.078164,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163631,-0.167106,-0.032993,...,1.709623,1.606604,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
203763,-0.093732,-0.116160,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.082559,-0.089510,-0.049707,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792


In [150]:
test_node_features_without_class = test_node_features.drop(columns=['class'])
test_node_features_without_class
# test_node_features_without_class.reset_index()[test_node_features_without_class.reset_index()['index'] == 45]




,2,3,4,5,6,7,8,9,10,11,...,157,158,159,160,161,162,163,164,165,166
0,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,-0.049707,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,-0.049707,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,-0.049707,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
4,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,0.935886,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117
5,0.961040,-0.081127,-1.201369,1.303743,0.333276,1.480381,-0.061584,-0.163577,0.038305,0.816377,...,-0.504702,-0.422589,-0.226790,-0.117629,0.018279,0.277775,0.413931,1.149556,-0.696053,-0.695540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203762,-0.172974,-0.156732,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163636,-0.169451,-0.049707,...,-0.577099,0.647874,0.241128,0.241406,7.165536,1.085202,-0.131155,5.157442,-0.120613,-0.119792
203764,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,-0.142008,-0.049707,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
203765,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,-0.162334,-0.049707,...,0.162722,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984
203767,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,-0.169317,-0.049707,...,-0.397749,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399


In [152]:
# now we need to reindex the train nodes and train edges so that node id is equal to row id

# train_node_features_without_class.reset_index()[train_node_features_without_class.reset_index()['index'] == 45]

In [161]:
reindex_dictionary_2 = dict()
for i, node_id in enumerate(train_node_features_without_class.reset_index()['index']):
  # print(i)
  # print(node_id)
  # print()
  reindex_dictionary_2[node_id] = i
  # if i > 115: break

# reindex_dictionary_2

In [162]:
train_edges

,0,1
0,10,11
1,16,17
2,30,31
3,52,53
4,66,67
...,...,...
36619,202182,201458
36620,202183,201342
36621,201882,203604
36622,203604,201331


In [164]:
train_edges_reindexed = train_edges.copy()
train_edges_reindexed[0] = train_edges[0].map(lambda node_id: reindex_dictionary_2[node_id])
train_edges_reindexed[1] = train_edges[1].map(lambda node_id: reindex_dictionary_2[node_id])
train_edges_reindexed

,0,1
0,2,3
1,4,5
2,9,10
3,12,13
4,19,20
...,...,...
36619,46255,46117
36620,46256,46094
36621,46199,46532
36622,46532,46089


In [166]:
# now crafting the input data structures to pytorch geometric Data object
edge_index = torch.tensor(np.array(train_edges_reindexed), dtype=torch.long)
edge_index

tensor([[    2,     3],
        [    4,     5],
        [    9,    10],
        ...,
        [46199, 46532],
        [46532, 46089],
        [46531, 46112]])

In [167]:
x = torch.tensor(np.array(train_node_features_without_class), dtype=torch.float)

In [223]:
# y = torch.tensor(np.array(train_classes))
y = torch.tensor(np.array(train_classes).reshape(len(train_classes), -1), dtype=torch.float)
y

tensor([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [1.],
        [0.]])

In [224]:
data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
data.validate(raise_on_error = True)

True

In [225]:
data

Data(x=[46564, 165], edge_index=[2, 36624], y=[46564, 1])

In [226]:
import random
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[random.sample(range(data.num_nodes), int(data.num_nodes * 0.8))] = 1
data.train_mask


tensor([ True,  True,  True,  ...,  True,  True, False])

In [227]:
from torch_geometric.nn import GCNConv

In [248]:
class Net(torch.nn.Module):
    def __init__(self, num_features):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 300)
        self.conv2 = GCNConv(300, 200)
        self.conv3 = GCNConv(200, 100)
        self.conv4 = GCNConv(100, 50)
        self.conv5 = GCNConv(50, 1)

        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)  
        x = self.conv2(x, edge_index)
        x = F.relu(x)  
        x = self.conv3(x, edge_index)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv4(x, edge_index)
        x = F.relu(x)  
        x = self.conv5(x, edge_index)
        x = self.sigmoid(x)
        return x


model = Net(data.num_features)

criterion = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [249]:
print(data.train_mask)
print(data.train_mask.shape)
data.x.shape

tensor([ True,  True,  True,  ...,  True,  True, False])
torch.Size([46564])


torch.Size([46564, 165])

In [254]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


In [259]:
# for epoch in range(30):
for epoch in range(30):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index).squeeze()
        pred = (pred > 0.5).float()  # Convert probabilities to binary predictions
        acc = (pred[data.train_mask] == data.y[data.train_mask]).float().mean()
    
    print(f'Epoch {epoch}: Train Accuracy: {acc.item():.4f}')



    # pred = model(data.x, data.edge_index).argmax(dim=1)
    # acc = (pred[data.train_mask] == data.y[data.train_mask]).float().mean()
    
    # print(f'Epoch {epoch}: Train Accuracy: {acc:.4f}')


Epoch 0: Train Accuracy: 0.9016
Epoch 1: Train Accuracy: 0.9016
Epoch 2: Train Accuracy: 0.9016
Epoch 3: Train Accuracy: 0.9016
Epoch 4: Train Accuracy: 0.9016
Epoch 5: Train Accuracy: 0.9016
Epoch 6: Train Accuracy: 0.9016


KeyboardInterrupt: ignored